# 卷積神經網路實作

## 載入套件

In [1]:
import tensorflow as tf
import input_data

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## 訓練資料

In [2]:
batch_xs, batch_ys = mnist.train.next_batch(5)

In [3]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

## 建立Computational Graph

In [4]:
x_ = tf.placeholder(tf.float32, [None, 784], name="x_")
y_ = tf.placeholder(tf.float32, [None, 10], name="y_")


x_image = tf.reshape(x_, [-1,28,28,1])

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y= tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


cross_entropy = -tf.reduce_sum(y_ * tf.log(y))

optimizer = tf.train.AdamOptimizer(1e-4)
trainer = optimizer.minimize(cross_entropy)

init = tf.initialize_all_variables()


## Session

In [5]:
sess = tf.Session()
sess.run(init)

## conv1

In [6]:
result_h_conv1 = sess.run(h_conv1,  feed_dict={x_: batch_xs})
result_h_conv1.shape

(5, 28, 28, 32)

## pool1

In [7]:
result_h_pool1 = sess.run(h_pool1,  feed_dict={x_: batch_xs})
result_h_pool1.shape

(5, 14, 14, 32)

## conv2

In [8]:
result_h_conv2 = sess.run(h_conv2,  feed_dict={x_: batch_xs})
result_h_conv2.shape

(5, 14, 14, 64)

## pool2

In [9]:
result_h_pool2 = sess.run(h_pool2,  feed_dict={x_: batch_xs})
result_h_pool2.shape

(5, 7, 7, 64)

## fc1

In [10]:
result_h_fc1 = sess.run(h_fc1,  feed_dict={x_: batch_xs})
result_h_fc1.shape

(5, 1024)

## y

In [11]:
result_y = sess.run(y,  feed_dict={x_: batch_xs, keep_prob:0.5})
result_y.shape

(5, 10)

## 建立成效評估用的Computational Graph

In [12]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Saver

In [13]:
saver = tf.train.Saver()


## 訓練

In [14]:
patience = 50
best_accurarcy = 0
i = 0


while True:
    i += 1
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(trainer,  feed_dict={x_: batch_xs, y_: batch_ys, keep_prob:0.5})
    if i%5 == 0:
        train_accurarcy = sess.run(accuracy, 
            feed_dict={x_: mnist.validation.images, y_: mnist.validation.labels, keep_prob:0.5})
        valid_accurarcy = sess.run(accuracy, 
            feed_dict={x_: mnist.validation.images, y_: mnist.validation.labels, keep_prob:1})
        print "%s, train:%s, valid:%s" %(i,train_accurarcy, valid_accurarcy)
        if valid_accurarcy > best_accurarcy:
            patience = 50
            best_accurarcy = valid_accurarcy
            print "save model"
            saver.save(sess, "model_conv.ckpt")
        else:
            patience -= 1
            if patience == 0:
                print "early stop"
                break

5, train:0.1206, valid:0.1242
save model
10, train:0.159, valid:0.2792
save model
15, train:0.2164, valid:0.4746
save model
20, train:0.2692, valid:0.552
save model
25, train:0.308, valid:0.6418
save model
30, train:0.3472, valid:0.6826
save model
35, train:0.4252, valid:0.714
save model
40, train:0.4678, valid:0.7478
save model
45, train:0.5132, valid:0.7708
save model
50, train:0.535, valid:0.79
save model
55, train:0.5682, valid:0.799
save model
60, train:0.5974, valid:0.8146
save model
65, train:0.6268, valid:0.8108
70, train:0.6398, valid:0.8392
save model
75, train:0.6546, valid:0.8334
80, train:0.6768, valid:0.8382
85, train:0.6856, valid:0.852
save model
90, train:0.707, valid:0.861
save model
95, train:0.7246, valid:0.8696
save model
100, train:0.7378, valid:0.8776
save model
105, train:0.7564, valid:0.8852
save model
110, train:0.7578, valid:0.889
save model
115, train:0.7668, valid:0.8934
save model
120, train:0.7664, valid:0.8924
125, train:0.7892, valid:0.8954
save model
1

In [15]:
saver.restore(sess, "model_conv.ckpt")
valid_accurarcy = sess.run(accuracy, feed_dict={x_: mnist.validation.images, y_: mnist.validation.labels, keep_prob:1})
test_accurarcy = sess.run(accuracy, feed_dict={x_: mnist.test.images, y_: mnist.test.labels, keep_prob:1})
print "valid:%s, test:%s"%(valid_accurarcy,test_accurarcy)

valid:0.9846, test:0.9828
